In [5]:
import ordered_set
from solana.rpc.api import Client
from solana.publickey import PublicKey
import time
import base64
import argparse

In [6]:
# open_orders.py
class OpenOrdersAccount: 
    def __init__(self, bytes):
        # Remove "serum" and "padding"
        bytes = bytes[5:-7]
        self.account_flags = int.from_bytes(bytes[0:8], "little")
        self.market = []
        for i in range(32):
            self.market.append(int.from_bytes(bytes[8+i:9+i], "little"))
        self.owner = PublicKey(bytes[48:72])
        self.native_base_free = int.from_bytes(bytes[72:80], "little")
        self.native_base_total = int.from_bytes(bytes[80:88], "little")
        self.native_quote_free = int.from_bytes(bytes[88:96], "little")
        self.native_quote_total = int.from_bytes(bytes[96:104], "little")
        self.free_slot_bits = int.from_bytes(bytes[104:120], "little")
        self.is_bid_bits = int.from_bytes(bytes[120:136], "little")
        self.orders = []
        for i in range(128):
            starting_index = 136 + i*16
            ending_index = 136 + (i + 1) * 16
            self.orders.append(
                int.from_bytes(bytes[starting_index:ending_index], "little")
            )
        self.client_order_ids = []
        for i in range(128):
            starting_index = 2184 + i*8
            ending_index = 2184 + (i + 1) * 8
            self.orders.append(
                int.from_bytes(bytes[starting_index:ending_index], "little")
            )
        self.referrer_rebates_accrued = int.from_bytes(bytes[3208:3216], "little")
    def get_owner(self):
        return self.owner

In [21]:
# event_queue.py
class EventQueueHeader: 
    def __init__(self, bytes):
        self.account_flags = int.from_bytes(bytes[0:8], "little")
        self.head = int.from_bytes(bytes[8:16], "little")
        self.count = int.from_bytes(bytes[16:24], "little")
        self.seq_num = int.from_bytes(bytes[24:32], "little")
    def __str__(self):
        return (
            "Account Flags: " + str(self.account_flags) + "\n" +
            "Head: " + str(self.head) + "\n" +
            "Count: " + str(self.count) + "\n" + 
            "Sequence Number: " + str(self.seq_num) + "\n" 
        )

class Event:
    def __init__(self, bytes):
        self.event_flags = int.from_bytes(bytes[0:1], "little")
        self.owner_slot = int.from_bytes(bytes[1:2], "little")
        self.fee_tier = int.from_bytes(bytes[2:3], "little")
        padding = []
        for i in range(5):
            padding.append(int.from_bytes(bytes[3+i:4+i], "little"))
        self._padding = padding
        self.native_quantity_released = int.from_bytes(bytes[8:16], "little")
        self.native_quantity_paid = int.from_bytes(bytes[16:24], "little")
        self.native_fee_or_rebate = int.from_bytes(bytes[24:32], "little")
        self.order_id = int.from_bytes(bytes[32:48], "little")
        self.owner = PublicKey(bytes[48:80])
        self.client_order_id = int.from_bytes(bytes[80:], "little")
    def __str__(self):
        return (
            "Event Flags: " + str(self.event_flags) + "\n" + 
            "Owner Slot: " + str(self.owner_slot) + "\n" + 
            "Fee Tier: " + str(self.fee_tier) + "\n" + 
            "Padding: " + ', '.join(str(item) for item in self._padding) + "\n" + 
            "Native Quantity Released: " + str(self.native_quantity_released) + "\n" +
            "Native Quantity Paid: " + str(self.native_quantity_paid) + "\n" + 
            "Native Fee or Rebate: " + str(self.native_fee_or_rebate) + "\n" + 
            "Order ID: " + str(self.native_fee_or_rebate) + "\n" + 
            "Owner: " + ', ' + self.owner + "\n" + 
            "Client Order ID: " + str(self.client_order_id) + "\n"
        )

class EventAccount:
    def __init__(self, bytes):
        bytes_without_padding = bytes[5:-7]
        header_bytes = bytes_without_padding[0:32]
        events_bytes = bytes_without_padding[32:]
        self.event_queue_header = EventQueueHeader(header_bytes)
        self.events = []
        for i in range(self.event_queue_header.count):
            starting_index = (self.event_queue_header.head + i) * 88
            ending_index = starting_index + 88
            self.events.append(Event(
                events_bytes[starting_index:ending_index]
            ))
            
    def print_overview(self):
        print(self.event_queue_header)
        print("Number of Events", len(self.events))
        
    def print_events(self):
        for i in range(self.event_queue_header.count):
            print("------------ Event Number", i+1, "----------")
            print(self.events[i])

In [22]:
def process_event(event, wallet_to_volume, client):
    # Get owner from Open Orders Account
    OPEN_ORDER = PublicKey(event.owner)
    account_info = client.get_account_info(OPEN_ORDER)
    account_data_raw = base64.b64decode(account_info['result']['value']['data'][0])
    open_orders_account = OpenOrdersAccount(account_data_raw)
    owner = open_orders_account.get_owner()
    
    # Process the owner's volume
    if owner not in wallet_to_volume:
        wallet_to_volume[owner] = 0
    #if event.event_flags % 2 == 1:
    wallet_to_volume[owner] += event.native_quantity_paid
    wallet_to_volume[owner] += event.native_quantity_released

In [23]:
grainularity = 2
total = 200

In [24]:
client = Client("https://api.mainnet-beta.solana.com")
EVENT_QUEUE = PublicKey("FTtrtEkcJaa84FRBwp7w5fUypBwvMsbNvS3KUD1HL97c")
messages = dict()
seen = ordered_set.OrderedSet()

In [25]:
for idx in range(total):
    account_info = client.get_account_info(EVENT_QUEUE)
    account_data_raw = base64.b64decode(account_info['result']['value']['data'][0])
    assert len(account_data_raw) == 262156
    if account_data_raw not in seen:
        messages[idx] = account_data_raw
        seen.add(account_data_raw)
    time.sleep(grainularity)

In [26]:
parsed_accounts = []
for raw_account in seen:
    parsed_accounts.append(EventAccount(raw_account))

num_parsed_accounts = len(parsed_accounts)
wallet_to_volume = {}
for i in range(num_parsed_accounts - 1):
    current_head = parsed_accounts[i].event_queue_header.head
    next_head = parsed_accounts[i+1].event_queue_header.head
    if current_head < next_head:
        difference = next_head - current_head 
    elif current_head == next_head:
        continue    
    else:
        difference = next_head + 2978 - current_head
    print("The current head, next head, and difference are", current_head, next_head, difference)

    current_count = parsed_accounts[i].event_queue_header.count
    for j in range(min(difference, current_count)):
        process_event(parsed_accounts[i].events[j], wallet_to_volume, client)
for event in parsed_accounts[-1].events:
    process_event(event, wallet_to_volume, client)
print(wallet_to_volume)

{}


In [28]:
len(parsed_accounts)

1

In [31]:
for account in parsed_accounts:
    print("?")
    account.print_overview()
    account.print_events()

?
Account Flags: 17
Head: 498
Count: 0
Sequence Number: 339990

Number of Events 0
